# 데이터변환 : 구간화, 정규화

In [1]:
import numpy as np
import pandas as pd

In [26]:
# 기사별 클릭수 데이터 셋 만들어서 추출하기 10000개
# category, journal, article_id, num_click
category_list = ['사설','사회','공학','증권','부동산','경제','스포츠','연예']
journal_list = ['A일보','B일보','C일보','D일보']

np.random.seed(2024)
# 카테고리 list
category = []
random_idx = np.random.randint(0,8,10000)
for i in random_idx:
    category.append(category_list[i])
# 저널 list
random_idx = np.random.randint(0,4,10000)
journal = []
for i in random_idx :
    journal.append(journal_list[i])
# article list
article_id = np.random.randn(10000)
# num_click list
num_click = np.random.randint(0,1000, size = 10000)

In [32]:
# 데이터 프레임 생성
df = pd.DataFrame({
    'category':category,
    'Journal' :journal,
    'article_id':article_id,
    'num_click':num_click
})
df['article_id'] = df['article_id'].astype(str)
df.to_csv('../data/click_sample_data.csv', sep=',', index=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   category    10000 non-null  object
 1   Journal     10000 non-null  object
 2   article_id  10000 non-null  object
 3   num_click   10000 non-null  int32 
dtypes: int32(1), object(3)
memory usage: 273.6+ KB


In [34]:
# 데이터 로딩
click_data = pd.read_csv('../data/click_sample_data.csv')
click_data.head(15)

,category,Journal,article_id,num_click
0,사설,B일보,-0.041786,819
1,공학,C일보,-1.892833,771
2,사설,C일보,-0.341424,804
3,사설,A일보,-0.756964,908
4,증권,D일보,-0.012348,458
5,부동산,A일보,0.426621,386
6,스포츠,A일보,-1.117491,703
7,연예,D일보,1.378980,154
8,사회,D일보,-1.057721,198
9,연예,A일보,-0.191078,252


In [36]:
# 데이터 개요 파악
click_data.info()

# 데이터 Copy
click_copy = click_data.copy()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   category    10000 non-null  object 
 1   Journal     10000 non-null  object 
 2   article_id  10000 non-null  float64
 3   num_click   10000 non-null  int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 312.6+ KB


### 구간화 (Binning)

#### 지정 길이 기반 구간

In [38]:
# 데이터 범위를 사용자의 기준대로 정의하고 구간화

In [39]:
click_copy.describe()

,article_id,num_click
count,10000.000000,10000.000000
mean,-0.001254,502.004500
std,1.002768,287.517351
min,-3.461941,0.000000
25%,-0.682761,257.000000
50%,-0.004626,504.500000
75%,0.673490,750.000000
max,3.757122,999.000000


* 기사 클릭 수가 20회인 기사외 10회인 기사 비교
* 20회 클릭된 기사가 2배 더 흥미있는 기사로 정의할 수 있는가?? <br>
-> 없음! 기사의 카테고리 저널 등 클릭을 유도하는 여러 요소가 있기 때문
* 따라서, 구간화를 통해 새로운 의미를 지닌 변수로 변환이 필요하며 목적 설정이 중요함
* 클릭 횟수를 기준으로 범주로 변환하여 각 범주 별 의미 설정 가능( 도메인 지식 검토 필요)

In [43]:
# 지정길이 기반 구간화
# 클릭 수 기반으로 기사 관심도 정의 기반 범주형 변환 (3개 구간 설정)

# cut 함수 활용 기반 구간 설정 (사용자 기준)
bins = [0, 100, 500, np.max(click_copy['num_click'])]
names = ['low', 'medium', 'high']

click_copy['pref'] = pd.cut(click_copy['num_click'], bins, labels=names)
# pd.cut(array, bins(나누고자 하는 방식), label) : 구간화 함수

click_copy.head()

,category,Journal,article_id,num_click,pref
0,사설,B일보,-0.041786,819,high
1,공학,C일보,-1.892833,771,high
2,사설,C일보,-0.341424,804,high
3,사설,A일보,-0.756964,908,high
4,증권,D일보,-0.012348,458,medium


In [45]:
# 구간 결과 확인
click_copy['pref'].value_counts()

# 임의로 구간을 정했기 때문에 구간마다 데이터 갯수가 다르다.

high      5046
medium    3942
low       1000
Name: pref, dtype: int64

#### 분포 기반 구간

In [46]:
# 각 데이터가 모든 구간 내 동일한 개수로 구분되도록 구간화

In [49]:
# 분포 기반 구간화
# qcut의 경우 동일한 관측치가 배치되도록 구간을 분할
# 따라서 구간 개수로 나누게 됨
n = 3 # 구간 수
click_copy['pref_qcut'] = pd.qcut(click_copy['num_click'], n, labels=names)
print(click_copy['pref_qcut'].value_counts())

# 정확히 3으로 나누어지는 구간길이가 아니기 떄문에 약간의 차이가 있지만
# 거의 동일한 관측치 개수를 가지고 있다

low       3342
medium    3329
high      3329
Name: pref_qcut, dtype: int64


#### qcut 실행 시 주로 발생하는 오류

* 데이터가 극단치의 Skewed 한 경우 발생
* 데이터의 각 구간은 동일할 수 없기 때문 <br>
 -> [1,1,1,1,1,3,8,10] 이라는 구간은 생성 불가능<br>
 -> qcut의 경우 나눌 구간의 개수만 입력하면 해당 구간 길이 즉, 범주를 알아서 정하하는데, 1이 5개나 있으므로 3등분의 경우 1 1 1, 1 1 3, 8 10 이렇게 나눠질 수가 없기 때문이다. 무조건 각 구간의 개수가 다를 수밖에 없으므로 불가능하다. 
* 그러한 경우에는 분포 기반 구간화(qcut)보다 사용자 기준 기반 범주화 혹은 구간 개수 변화 고려

### 정규화

#### 기계학습 알고리즘의 각 변수 별 영향력의 차이를 조정할 필요 존재함
* 각 변수 별 범위가 다를 경우 학습 안정성이 떨어질 수 있으므로 간격 조정 필요

1. 최대-최소 정규화 : 변수 안의 값을 [0,1] 구간의 값을 갖도록 구성하고 데이터 군 내에서 특정 데이터가 가지는 위치를 보고자 할 때 사용
2. z-score 정규화 : 특정 데이터가 평균을 기준으로 얼마나 떨어져 있는지 파악하고 측정 스케일이 다른 경우의 데이터를 비교하는데 사용

#### 최대-최소 정규화

In [55]:
# min-max scaling
# 변수 값을 0과 1사이로 변환
# 함수는 sklearn 라이브러리에 쉽게 구현되어 있음
from sklearn.preprocessing import MinMaxScaler

In [76]:
scaler = MinMaxScaler()

# 데이터 copy
click_copy = click_data.copy()
click_copy.loc[100,'num_click'] = 100000 # 실습위해서 극단치
# 변환, 학습 fitting과 변환을 한번에 진행
click_copy['minmax_values'] = scaler.fit_transform(click_copy[['num_click']])
display(click_copy.head(15))

# fit_transform(dataframe)
print(type(click_copy['num_click'])) # 1D array 이므로 Series
print(type(click_copy[['num_click']])) # 2D array 가 DataFrame

,category,Journal,article_id,num_click,minmax_values
0,사설,B일보,-0.041786,819,0.00819
1,공학,C일보,-1.892833,771,0.00771
2,사설,C일보,-0.341424,804,0.00804
3,사설,A일보,-0.756964,908,0.00908
4,증권,D일보,-0.012348,458,0.00458
5,부동산,A일보,0.426621,386,0.00386
6,스포츠,A일보,-1.117491,703,0.00703
7,연예,D일보,1.378980,154,0.00154
8,사회,D일보,-1.057721,198,0.00198
9,연예,A일보,-0.191078,252,0.00252


<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>


In [78]:
click_copy.describe()
# 100000이라는 극단치가 1이되므로 나머지 값들은 0에 가깝게 된다.
# 즉, min-max scaling은 이상치에 치명적이다.

,article_id,num_click,minmax_values
count,10000.000000,10000.000000,10000.000000
mean,-0.001254,511.955400,0.005120
std,1.002768,1035.688806,0.010357
min,-3.461941,0.000000,0.000000
25%,-0.682761,257.000000,0.002570
50%,-0.004626,505.000000,0.005050
75%,0.673490,750.000000,0.007500
max,3.757122,100000.000000,1.000000


### Z-score 정규화

In [80]:
# 표준화라고 불리기도 함
# 변수를 평균이 0이고 표준편차가 1인 표준정규분포로 변환
# 함수는 sklearn 라이브러리에 쉽게 구현되어 있음

from sklearn.preprocessing import StandardScaler
std_scaler = StandardScaler()

# 변환
click_copy['std_value'] = std_scaler.fit_transform(click_copy[['num_click']])
click_copy.head(101)

,category,Journal,article_id,num_click,minmax_values,std_value
0,사설,B일보,-0.041786,819,0.00819,0.296479
1,공학,C일보,-1.892833,771,0.00771,0.250131
2,사설,C일보,-0.341424,804,0.00804,0.281995
3,사설,A일보,-0.756964,908,0.00908,0.382416
4,증권,D일보,-0.012348,458,0.00458,-0.052099
...,...,...,...,...,...,...
96,경제,D일보,-2.876808,162,0.00162,-0.337913
97,경제,B일보,0.731964,608,0.00608,0.092740
98,사설,B일보,0.586011,876,0.00876,0.351518
99,부동산,C일보,-2.388531,665,0.00665,0.147778


In [81]:
# 데이터 요약
click_copy.describe()

,article_id,num_click,minmax_values,std_value
count,10000.000000,10000.000000,10000.000000,1.000000e+04
mean,-0.001254,511.955400,0.005120,-8.776313e-18
std,1.002768,1035.688806,0.010357,1.000050e+00
min,-3.461941,0.000000,0.000000,-4.943386e-01
25%,-0.682761,257.000000,0.002570,-2.461822e-01
50%,-0.004626,505.000000,0.005050,-6.716060e-03
75%,0.673490,750.000000,0.007500,2.298533e-01
max,3.757122,100000.000000,1.000000,9.606459e+01


### 어느 방안이 더 나은 것인가?
* 상황에 따라 달라지며, 정규화 수행과 수행하지 않은 결과의 차이가 큰 것이 중요한 포인트임
* 연속형 데이터를 기계학습 모델링에 적용하기 위해서 변수 간의 상대적 크기 차이를 제거할 필요가 반드시 존재함
* ex)<br>
    -> 대표적 비지도학습인 군집분석 중 Kmeans는 수치형 변수로 적용하는 알고리즘<br>
    -> 변수간 수치적 distance(거리)를 연산하여 유사한 집단으로 묶는 알고리즘<br>
    -> 변수가 각자 다른 범위를 가진 경우 유사 집단 구성 시, 변수 간 영향력의 차이가 반영되어 올바르지 못한 결과 도출 가능성 존재